In [17]:
from os import scandir, path
import pyspark
import pyspark.sql.functions as F
from pyspark.sql.functions import lit,create_map
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.window import Window

from itertools import chain

In [3]:
sc = SparkContext.getOrCreate()

In [4]:
sqlContext = pyspark.sql.SQLContext(sc)

In [5]:
indicesDF = sqlContext.read.format('csv') \
  .options(header='true', inferSchema='true') \
  .load('gs://financials-data-bucket/data/GSPC-2.csv')

In [6]:
indicesDF = indicesDF.select( \
F.col('Date').alias('DATE'), \
F.col('Open').alias('OPEN'), \
F.col('High').alias('HIGH'), \
F.col('Low').alias('LOW'),\
F.col('Close').alias('CLOSE'),\
F.col('Adj Close').alias('ADJ_CLOSE'),
F.col('Volume').alias('VOLUME'))

In [9]:
indicesDF = indicesDF.withColumn('YEAR', F.substring('DATE', 1, 4).cast(IntegerType()))
indicesDF = indicesDF.withColumn('INDICE', lit(1))
indicesDF.show()

+-------------------+---------+---------+---------+---------+---------+-------+----+------+
|               DATE|     OPEN|     HIGH|      LOW|    CLOSE|ADJ_CLOSE| VOLUME|YEAR|INDICE|
+-------------------+---------+---------+---------+---------+---------+-------+----+------+
|1950-01-03 00:00:00|    16.66|    16.66|    16.66|    16.66|    16.66|1260000|1950|     1|
|1950-01-04 00:00:00|    16.85|    16.85|    16.85|    16.85|    16.85|1890000|1950|     1|
|1950-01-05 00:00:00|    16.93|    16.93|    16.93|    16.93|    16.93|2550000|1950|     1|
|1950-01-06 00:00:00|    16.98|    16.98|    16.98|    16.98|    16.98|2010000|1950|     1|
|1950-01-09 00:00:00|    17.08|    17.08|    17.08|    17.08|    17.08|2520000|1950|     1|
|1950-01-10 00:00:00|17.030001|17.030001|17.030001|17.030001|17.030001|2160000|1950|     1|
|1950-01-11 00:00:00|    17.09|    17.09|    17.09|    17.09|    17.09|2630000|1950|     1|
|1950-01-12 00:00:00|    16.76|    16.76|    16.76|    16.76|    16.76|2970000|1

#### Nos quedamos con los datos a partir de 1970 ya que no tenemos datos anteriores en price

In [10]:
indicesDF = indicesDF.filter("Date > '1970-01-01'").sort(F.asc('Date'))
indicesDF.show()

+-------------------+---------+---------+---------+---------+---------+--------+----+------+
|               DATE|     OPEN|     HIGH|      LOW|    CLOSE|ADJ_CLOSE|  VOLUME|YEAR|INDICE|
+-------------------+---------+---------+---------+---------+---------+--------+----+------+
|1970-01-02 00:00:00|92.059998|93.540001|91.790001|     93.0|     93.0| 8050000|1970|     1|
|1970-01-05 00:00:00|     93.0|    94.25|92.529999|93.459999|93.459999|11490000|1970|     1|
|1970-01-06 00:00:00|93.459999|93.809998|92.129997|    92.82|    92.82|11460000|1970|     1|
|1970-01-07 00:00:00|    92.82|93.379997|    91.93|92.629997|92.629997|10010000|1970|     1|
|1970-01-08 00:00:00|92.629997|93.470001|91.989998|    92.68|    92.68|10670000|1970|     1|
|1970-01-09 00:00:00|    92.68|    93.25|    91.82|92.400002|92.400002| 9380000|1970|     1|
|1970-01-12 00:00:00|92.400002|92.669998|91.199997|91.699997|91.699997| 8900000|1970|     1|
|1970-01-13 00:00:00|91.699997|92.610001|90.989998|91.919998|91.919998

In [11]:
#Rentabilidad diaria
windowSpec = Window.orderBy(F.col("DATE")).rowsBetween(-1, 0)
indicesDF = indicesDF.withColumn('AUX', F.sum("ADJ_CLOSE").over(windowSpec))
indicesDF = indicesDF.withColumn("RETURNS", (F.col("ADJ_CLOSE") - (F.col("AUX")-F.col("ADJ_CLOSE"))) / F.col("ADJ_CLOSE")).drop("AUX")
indicesDF.show()

+-------------------+---------+---------+---------+---------+---------+--------+----+------+--------------------+
|               DATE|     OPEN|     HIGH|      LOW|    CLOSE|ADJ_CLOSE|  VOLUME|YEAR|INDICE|             RETURNS|
+-------------------+---------+---------+---------+---------+---------+--------+----+------+--------------------+
|1970-01-02 00:00:00|92.059998|93.540001|91.790001|     93.0|     93.0| 8050000|1970|     1|                 1.0|
|1970-01-05 00:00:00|     93.0|    94.25|92.529999|93.459999|93.459999|11490000|1970|     1|0.004921881071280672|
|1970-01-06 00:00:00|93.459999|93.809998|92.129997|    92.82|    92.82|11460000|1970|     1|-0.00689505494505...|
|1970-01-07 00:00:00|    92.82|93.379997|    91.93|92.629997|92.629997|10010000|1970|     1|-0.00205120378013...|
|1970-01-08 00:00:00|92.629997|93.470001|91.989998|    92.68|    92.68|10670000|1970|     1|5.395230902028894E-4|
|1970-01-09 00:00:00|    92.68|    93.25|    91.82|92.400002|92.400002| 9380000|1970|   

In [18]:
#Rentabilidad acumulada
wCY = Window.partitionBy("YEAR").orderBy("DATE")
#Nos quedamos con el precio de cada compañia en cada año el dia 1
dico = indicesDF.withColumn("RminD",  F.row_number().over(wCY)).filter("RminD == 1").drop("DATE", "RminD") 
#Lista del tipo codyear: precio inicial al principio de año
df_dict = [{str(r['YEAR']): r['ADJ_CLOSE']} for r in dico.orderBy("YEAR").collect()] #Es una lista
df_dict = dict((key,d[key]) for d in df_dict for key in d) #Transformo a diccionario
mapping_expr = create_map([F.lit(x) for x in chain(*df_dict.items())]) #Transformamos a un mapa
r_acuDF = indicesDF.withColumn("CUMULATIVE_RETURNS", (F.col("ADJ_CLOSE") - mapping_expr.getItem(F.col("YEAR").cast(StringType()))) / mapping_expr.getItem(F.col("YEAR").cast(StringType())))
r_acuDF.show(1000)

+-------------------+----------+----------+----------+----------+----------+--------+----+------+--------------------+--------------------+
|               DATE|      OPEN|      HIGH|       LOW|     CLOSE| ADJ_CLOSE|  VOLUME|YEAR|INDICE|             RETURNS|  CUMULATIVE_RETURNS|
+-------------------+----------+----------+----------+----------+----------+--------+----+------+--------------------+--------------------+
|1970-01-02 00:00:00| 92.059998| 93.540001| 91.790001|      93.0|      93.0| 8050000|1970|     1|                 1.0|                 0.0|
|1970-01-05 00:00:00|      93.0|     94.25| 92.529999| 93.459999| 93.459999|11490000|1970|     1|0.004921881071280672|0.004946225806451...|
|1970-01-06 00:00:00| 93.459999| 93.809998| 92.129997|     92.82|     92.82|11460000|1970|     1|-0.00689505494505...|-0.00193548387096...|
|1970-01-07 00:00:00|     92.82| 93.379997|     91.93| 92.629997| 92.629997|10010000|1970|     1|-0.00205120378013...|-0.00397852688172...|
|1970-01-08 00:00:00

In [19]:
# Escribimos fichero final
indicesDF.coalesce(1) \
.write.format("com.databricks.spark.csv") \
.option("header", "true") \
.save("gs://financials-data-bucket/data/prueba/indices.csv")